In [133]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.callbacks import EarlyStopping
import random
from keras import Input
from keras.layers import Dropout
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau

In [134]:
# Cargar los datos
df = pd.read_csv('../datos_simulados/datos_final.csv')
# Variables categóricas importantes
#columnas_categoricas = ['Clase', 'Pieza', 'Superficie_1', 'Superficie_2', 'Estado_cavidad', 'Tecnica']
columnas_categoricas = ['Clase', 'Pieza', 'Superficie_1', 'Superficie_2', 'Tecnica']
# Variables numéricas relevantes
columnas_numericas = ['Tamanio_cavidad_mm', 'Indice_contraccion_%', 'Margen_cavo_mm']

# Codificar variables categóricas
df_codificado = pd.get_dummies(df[columnas_categoricas + columnas_numericas])

# Definir X (entradas) e y (salida)
X = df_codificado
y = df['Peso_resina_inicial_mg'] - df['Peso_sobrante_mg']

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [135]:
# Normalizar características con StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [136]:
# Crear el modelo secuencial
modelo_mlp = Sequential([
    Input(shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
])  # salida para regresión

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.5, min_lr=1e-6)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Compilar
modelo_mlp.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Entrenar
historial = modelo_mlp.fit(
    X_train_scaled, y_train,
    epochs=150,
    batch_size=16,
    validation_split=0.2,
    verbose=1,
    #callbacks=[early_stop, reduce_lr]
)

Epoch 1/150
744/744 ━━━━━━━━━━━━━━━━━━━━ 1s 781us/step - loss: 2197.9241 - mae: 33.3491 - val_loss: 262.1385 - val_mae: 13.6663
Epoch 2/150
744/744 ━━━━━━━━━━━━━━━━━━━━ 1s 711us/step - loss: 263.5393 - mae: 13.6193 - val_loss: 264.0851 - val_mae: 13.7257
Epoch 3/150
744/744 ━━━━━━━━━━━━━━━━━━━━ 1s 677us/step - loss: 259.6660 - mae: 13.5473 - val_loss: 265.3530 - val_mae: 13.6948
Epoch 4/150
744/744 ━━━━━━━━━━━━━━━━━━━━ 1s 788us/step - loss: 253.9429 - mae: 13.3483 - val_loss: 272.5323 - val_mae: 13.7591
Epoch 5/150
744/744 ━━━━━━━━━━━━━━━━━━━━ 1s 680us/step - loss: 259.4663 - mae: 13.5388 - val_loss: 260.8682 - val_mae: 13.6400
Epoch 6/150
744/744 ━━━━━━━━━━━━━━━━━━━━ 0s 605us/step - loss: 256.3923 - mae: 13.4559 - val_loss: 260.5898 - val_mae: 13.6930
Epoch 7/150
744/744 ━━━━━━━━━━━━━━━━━━━━ 1s 738us/step - loss: 257.9910 - mae: 13.4661 - val_loss: 262.3890 - val_mae: 13.6444
Epoch 8/150
744/744 ━━━━━━━━━━━━━━━━━━━━ 1s 676us/step - loss: 257.3615 - mae: 13.5011 - val_loss: 259.5949 - 

In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_mse, test_mae = modelo_mlp.evaluate(X_test_scaled, y_test, verbose=0)
print(f"MAE en prueba: {test_mae:.2f}")
print(f"MSE en prueba: {test_mse:.2f}")

In [ ]:
# Recuperar la pérdida por época durante el entrenamiento
train_loss_history = historial.history['loss']
epochs = range(len(train_loss_history))

# Gráfica general de pérdida
plt.plot(epochs, train_loss_history, 'r')
plt.title('Training Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss (MSE)")
plt.legend(["Loss"])
plt.grid(True)
plt.show()

In [ ]:
# Zoom a partir de la época 20
zoomed_loss = train_loss_history[20:]
zoomed_epochs = range(20, len(train_loss_history))

plt.plot(zoomed_epochs, zoomed_loss)
plt.title("Zoomed Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss (MSE)")
plt.grid(True)
plt.show()

In [ ]:
# Hacer predicciones sobre el conjunto de prueba
y_pred = modelo_mlp.predict(X_test_scaled)

# Calcular RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calcular R²
r2 = r2_score(y_test, y_pred)

# Mostrar resultados
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")


In [ ]:
# Calcular la media del valor real de la resina inicial
media_real = df["Peso_resina_inicial_mg"].mean()

# RMSE obtenido tras evaluación del modelo
rmse = rmse  # Sustituye por el valor real si cambia

# Calcular el error relativo porcentual
error_relativo_pct = (rmse / media_real) * 100
precision_aproximada = 100 - error_relativo_pct

# Mostrar resultados
print(f"Media real de peso de resina inicial: {media_real:.2f} mg")
print(f"RMSE: {rmse:.2f} mg")
print(f"Error relativo aproximado: {error_relativo_pct:.2f} %")
print(f"Precisión aproximada del modelo: {precision_aproximada:.2f} %")

In [ ]:
# Diccionario de índice de contracción
indice_contraccion = {
    "Compuesta": 2.0,
    "Fluida": 3.5,
    "Con fibra de vidrio": 1.0,
    "Bulk": 1.5
}

# Configuraciones posibles
tecnicas = ["Bulk", "Estratificacion"]
tipos_resina = list(indice_contraccion.keys())
ajustes_margen = ["Sobreobturado", "Subobturado", "Equiobturado"]
estados_cavidad = ["con_tejido", "limpia"]

# Leer combinaciones base
df_base = pd.read_excel("../datos_simulados/Combinaciones.xlsx")

# Lista para una combinación por fila
combinaciones_X = []

for _, fila in df_base.iterrows():
    tecnica = random.choice(tecnicas)
    tipo_resina = random.choice(tipos_resina)
    indice = indice_contraccion[tipo_resina]

    tam = round(random.uniform(fila['Tam_min'], fila['Tam_max']), 2)

    ajuste = random.choice(ajustes_margen)
    if ajuste == "Equiobturado":
        margen = round(random.uniform(tam - 0.2, tam + 0.2), 2)
    elif ajuste == "Sobreobturado":
        margen = round(random.uniform(tam + 0.01, tam + 1.0), 2)
    else:
        margen = round(random.uniform(max(0.1, tam - 1.0), tam - 0.01), 2)

    combinaciones_X.append({
        "Clase": fila["Clase"],
        "Pieza": fila["Pieza"],
        "Superficie_1": fila["Superficie_1"],
        "Superficie_2": fila["Superficie_2"],
        "Estado_cavidad": random.choice(estados_cavidad),
        "Tecnica": tecnica,
        "Tamanio_cavidad_mm": tam,
        "Indice_contraccion_%": indice,
        "Margen_cavo_mm": margen
    })

# Crear DataFrame con combinaciones generadas
df_combos = pd.DataFrame(combinaciones_X)

# Codificar y alinear columnas como en el entrenamiento
df_combos_codificado = pd.get_dummies(df_combos)
df_combos_codificado = df_combos_codificado.reindex(columns=X_train.columns, fill_value=0)

# Aplicar el mismo escalado
df_combos_scaled = scaler.transform(df_combos_codificado)

# Predecir la resina inicial
resina_recomendada = modelo_mlp.predict(df_combos_scaled)

# Mostrar resultados
for i, valor in enumerate(resina_recomendada.flatten(), start=1):
    datos = df_combos.loc[i - 1]
    print(f"Combinación {i} → {valor:.2f} mg | Cavidad: {datos['Tamanio_cavidad_mm']} mm | Clase: {datos['Clase']}")
